# Imports

In [1]:
import itertools
import numpy as np
import pandas as pd
import networkx as nx

# Config

In [2]:
# board size = length of one side of hexagonal board
board_size = 4

# Create Board

* hexagonal 2d space can be represented with 3 axes on a 2d plane equally 
separated
* each integer coordinate is a single square on the board
* neighbours are then +1 in each axes, maxed at the size of the board

In [81]:
nodes = np.array(
    list(itertools.product(*[range(-board_size + 1, board_size)] * 2))
)
directions = list(itertools.product(*[[-1, 0, 1]] * 2))
directions.pop(directions.index((0, 0)))
directions = np.array(directions)

In [82]:
edges = {}
for i, node in enumerate(nodes):
    edges[i] = []
    for direction in directions:
        neighbour = node + direction
        if (np.abs(neighbour) >= board_size).sum() > 0:
            continue
        neighbour_index = nodes.tolist().index(neighbour.tolist())
        edges[i].append(neighbour_index)
g = nx.from_dict_of_lists(edges)

In [88]:
print(f"from node:\n{nodes[0]}\n")
print("to nodes:")
for to_node in g[0].keys():
    print(nodes[to_node])

from node:
[-3 -3]

to nodes:
[-3 -2]
[-2 -3]
[-2 -2]


In [77]:
nodes[0]

array([-3, -3, -3])

In [78]:
4 + 5 + 6 + 7 + 6 + 5 + 4

37

In [84]:
len(nodes)

49

---

In [3]:
def hexagonal_coordinates(n):
    """Generate all coordinates for a hexagonal board with side length n."""
    for x in range(-n + 1, n):
        for y in range(max(-n + 1, -x - n + 1), min(n, -x + n)):
            z = -x - y
            yield (x, y, z)


def hexagonal_neighbors(x, y, z):
    """Yield the neighbors of a given hexagon cell in cube coordinates."""
    # Six possible directions for a hex cell
    directions = [
        (+1, -1, 0),
        (+1, 0, -1),
        (0, +1, -1),
        (-1, +1, 0),
        (-1, 0, +1),
        (0, -1, +1),
    ]
    for dx, dy, dz in directions:
        yield (x + dx, y + dy, z + dz)


def create_hexagonal_board(n):
    """Create a DataFrame representing the edges of a hexagonal board."""
    coords = list(hexagonal_coordinates(n))
    edges = []
    seen = set()

    for x, y, z in coords:
        for nx, ny, nz in hexagonal_neighbors(x, y, z):
            if (nx, ny, nz) in coords:
                edge = tuple(sorted([(x, y, z), (nx, ny, nz)]))
                if edge not in seen:
                    seen.add(edge)
                    edges.append(edge)

    # Creating DataFrame from edges list
    edge_df = pd.DataFrame(edges, columns=["source", "target"])
    return edge_df

In [4]:
# Example usage
n = 2  # Side length of the hexagonal board
df_edges = create_hexagonal_board(n)
print(df_edges)

        source      target
0   (-1, 0, 1)  (0, -1, 1)
1   (-1, 0, 1)   (0, 0, 0)
2   (-1, 0, 1)  (-1, 1, 0)
3   (-1, 1, 0)   (0, 0, 0)
4   (-1, 1, 0)  (0, 1, -1)
5   (0, -1, 1)  (1, -1, 0)
6   (0, -1, 1)   (0, 0, 0)
7    (0, 0, 0)  (1, -1, 0)
8    (0, 0, 0)  (1, 0, -1)
9    (0, 0, 0)  (0, 1, -1)
10  (0, 1, -1)  (1, 0, -1)
11  (1, -1, 0)  (1, 0, -1)


In [5]:
g = nx.from_pandas_edgelist(df_edges)

In [6]:
list(g.nodes())

[(-1, 0, 1),
 (0, -1, 1),
 (0, 0, 0),
 (-1, 1, 0),
 (0, 1, -1),
 (1, -1, 0),
 (1, 0, -1)]

In [ ]:
"""
.|0|0|.
|0|0|0|
.|0|0|.
"""

In [ ]:
"""
(-1, 0, 1)  ->  (-1,1)
(0, -1, 1)  ->  (1,1)
(0, 0, 0)   ->  (0,0)
(-1, 1, 0)  ->  (0,-1)
(0, 1, -1)  ->  (-1,-1)
(1, -1, 0)  ->  (0,1)
(1, 0, -1)  ->  (1,-1)
"""

In [8]:
def cubic_to_pixel(x, y, z):
    return (x - y, -x - y)

In [13]:
pixel_coordinates = list(map(lambda x: cubic_to_pixel(*x), g.nodes()))
for cn, pn in zip(g.nodes(), pixel_coordinates):
    print(f"{cn}\t->\t{pn}")

(-1, 0, 1)	->	(-1, 1)
(0, -1, 1)	->	(1, 1)
(0, 0, 0)	->	(0, 0)
(-1, 1, 0)	->	(-2, 0)
(0, 1, -1)	->	(-1, -1)
(1, -1, 0)	->	(2, 0)
(1, 0, -1)	->	(1, -1)


In [35]:
def print_board(graph, size):
    cubic_nodes = graph.nodes()
    pixel_nodes = list(map(lambda x: cubic_to_pixel(*x), cubic_nodes))

    sorted_pixel_nodes = sorted(
        pixel_nodes, key=lambda x: (x[1], -x[0]), reverse=True
    )

    board = np.full((size + 1, size + 1), fill_value=".")

    for node in pixel_nodes:
        x, y = node[0]
        board[node] = 0

    for i in range(size):
        for j in range(size):
            print(board[i][j])
        print()
    print(board)

In [36]:
print_board(g, 2)

0
.

0
0

[['0' '.' '.']
 ['0' '0' '0']
 ['0' '0' '0']]
